In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from PIL import Image
import tensorflow as tf

from tensorflow import  keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
import tensorflow_datasets as tfds
import pickle
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import random
import glob
from tensorflow.keras.preprocessing.image import img_to_array
from keras_preprocessing.image.utils import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_dir= './File'

In [ ]:
train_data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="training",
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)
val_data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="validation",
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

'train_data = tf.keras.utils.image_dataset_from_directory(\n    data_dir,\n    labels=\'inferred\',\n    label_mode=\'int\',\n    class_names=None,\n    color_mode=\'rgb\',\n    batch_size=32,\n    image_size=(256, 256),\n    shuffle=True,\n    seed=123,\n    validation_split=0.2,\n    subset="training",\n    interpolation=\'bilinear\',\n    follow_links=False,\n    crop_to_aspect_ratio=False,\n)\nval_data = tf.keras.utils.image_dataset_from_directory(\n    data_dir,\n    labels=\'inferred\',\n    label_mode=\'int\',\n    class_names=None,\n    color_mode=\'rgb\',\n    batch_size=32,\n    image_size=(256, 256),\n    shuffle=True,\n    seed=123,\n    validation_split=0.2,\n    subset="validation",\n    interpolation=\'bilinear\',\n    follow_links=False,\n    crop_to_aspect_ratio=False,\n)'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
batch_size=2
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)
val_datagen = ImageDataGenerator(
        rescale=1./255
        )

train_generator = train_datagen.flow_from_directory(
        data_dir,  # this is the target directory
        target_size=(256, 256),  # all images will be resized to 256x256
        batch_size=batch_size,
        shuffle=True,
        class_mode='categorical',
        subset="training",
        interpolation='bilinear',
        follow_links=False,
        )
val_generator = train_datagen.flow_from_directory(
        data_dir,  # this is the target directory
        target_size=(256, 256),  # all images will be resized to 256x256
        batch_size=batch_size,
        shuffle=True,
        class_mode='categorical',
        subset="validation",
        interpolation='bilinear',
        follow_links=False,
        )

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [ ]:
class_names = train_generator.class_indices
class_num =len(class_names)
print(class_names, class_num)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4} 5


In [ ]:
train_generator.labels

array([0, 0, 0, ..., 4, 4, 4], dtype=int32)

In [ ]:
next(train_generator)[0].shape

(2, 256, 256, 3)

In [ ]:
input_shape = (256,256,3)
_model_denseNet = tf.keras.applications.vgg16.VGG16(include_top = False, input_shape=input_shape,classes=class_num)

for layers in _model_denseNet.layers:
  layers.trainable=False

pooling = tf.keras.layers.MaxPool2D()(_model_denseNet.output)
feature = tf.keras.layers.Dense(128,activation="relu")(pooling)
feature = tf.keras.layers.Flatten()(feature)
feature = tf.keras.layers.Dense(class_num,activation="softmax")(feature)
model_denseNet = tf.keras.Model(_model_denseNet.input, feature)

model_denseNet.compile(optimizer='adam',
              loss= tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model_denseNet.summary()

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0  

In [ ]:
epochs = 5
history = model_denseNet.fit_generator( train_generator, epochs = epochs, validation_data = val_generator, shuffle=1 )